# **<span style="font-family: 'Palatino Linotype', serif;"> 🛡️ Preparando-se para a batalha!</span>**

*<span style="font-family: 'Angilla Tattoo'">A jornada mais longa começa com um único passo...</span>*

<div align="center">
    <img src="assets\mapas\1.png" alt="Mapa: A Batalha Final">
    <figcaption>Guerreiros rumo ao Vilarejo Vizinhos</figcaption>
</div>

---

**Trabalho Final: 🧑‍🎓 Introdução - Apresentando o trabalho**
==========================================================

**Autores:** Sepulcro de Delfos
* Ana Luz 
* Caio Ruas
* Caio Matheus
* Giovana Martins

## 🚩 **Introdução**

A crescente demanda por energia limpa e renovável impulsionou o desenvolvimento de tecnologias de energia solar, com destaque para as células fotovoltaicas. Essas células, capazes de converter luz solar diretamente em eletricidade, desempenham um papel crucial na transição para uma matriz energética mais sustentável. O desempenho de uma célula fotovoltaica é fortemente influenciado pelas propriedades dos materiais que a compõem, particularmente pelo *band gap*, uma propriedade fundamental que determina a eficiência na conversão de fótons em elétrons.

Ele representa a diferença de energia entre a banda de valência, onde os elétrons estão ligados aos átomos, e a banda de condução, onde os elétrons estão livres para se mover e gerar corrente elétrica. A importância do *band gap* para as células fotovoltaicas reside em sua influência direta na capacidade de conversão da luz solar em energia elétrica. Para que um elétron seja liberado de um átomo e contribua para a corrente elétrica, ele precisa receber uma quantidade mínima de energia, que corresponde ao valor do band gap[1].

A busca por materiais com propriedades otimizadas para aplicação em células fotovoltaicas é um desafio constante na ciência dos materiais. A descoberta de novos materiais e a otimização de materiais existentes exigem a análise de um vasto espaço de candidatos, o que pode ser acelerado com o uso de técnicas de aprendizado de máquina.

Nesse contexto, o **The Materials Project**[2][3] se demonstra uma importante base de dados para informações sobre materiais. Essa iniciativa disponibiliza um banco de dados abrangente com informações sobre as propriedades de milhares de materiais, incluindo suas estruturas cristalinas, propriedades eletrônicas e termodinâmicas. Essa riqueza de informações oferece uma oportunidade única para o desenvolvimento de modelos de aprendizado de máquina capazes de prever propriedades de materiais com base em atributos estruturais e químicos.

Este trabalho final propõe a utilização do banco de dados do *Materials Project* e técnicas de aprendizado de máquina para investigar a relação entre as propriedades dos materiais e o *band gap*. O objetivo é treinar um algoritmo capaz de prever o *band gap* de materiais com base em um conjunto de atributos e estudar dois tipos diferentes de algoritmos, sendo eles o **KNN (K-nearest neighbors)** e a **Árvore de decisões**. A aplicação de aprendizado de máquina nesse contexto pode acelerar a descoberta de novos materiais e otimizar o desenvolvimento de células fotovoltaicas mais eficientes, impulsionando a geração de energia solar e a transição para uma matriz energética mais limpa e sustentável.

**Observação**: Durante todos os notebooks foi utilizada a biblioteca plotly para a criação de gráficos. Entretanto, no github, os gráficos não são renderizados. Para visualizá-los, basta executar o notebook em sua máquina local ou acessar o link das visualizações disponibilizado sempre que houver um gráfico.

## 🌿 **Preparando o ambiente**

Bibliotecas necessárias para a execução do código:

In [4]:
import json
import os
import re

import optuna
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from emmet.core.summary import HasProps
from mp_api.client import MPRester
from pymatgen.core.periodic_table import Element
from scipy.stats.mstats import winsorize
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



optuna.logging.set_verbosity(optuna.logging.WARNING) # Essa linha de código suprime os logs do optuna, facilitando a leitura do notebook.

Caso não esteja familiarizado com alguma das bibliotecas acima, não se preocupe, pois a maioria é explicada ao longo do código. Além disso, todas as documentações podem ser encontradas ao fim do notebook.

Funções que serão utilizadas ao longo do caderno:

In [5]:
def calc_fracao_molar(composicao: str) -> dict:
  """
  Calcula as frações molares de cada elemento presente em uma composição.

  Args:
    composicao (`str`): string de composição, por exemplo, "Ba2 Pd4 S8".

  Returns:
    fractions `dict`: Um dicionário com as frações de cada elemento.
  """

  elements = re.findall(r"([A-Z][a-z]*)(\d*)", composicao)
  total_atoms = 0
  counts = {}
  for element, count in elements:
      count = int(count) if count else 1
      counts[element] = count
      total_atoms += count
  fractions = {element: count / total_atoms for element, count in counts.items()}
  return fractions

In [6]:
def numero_atomico_medio(fracoes: dict) -> float:
    """
    Calcula o número atômico médio de uma dada composição.

    Args:
        fracoes (`dict`): dicionário com as frações molares de cada elemento presente na composição.

    Returns:
        numero_atomico (`float`): O número atômico médio da composição. 
    """
    numero_atomico = 0
    for elemento, fracao in fracoes.items():
        elemento_obj = Element(elemento)  # Cria um objeto Element do pymatgen
        numero_atomico += fracao * elemento_obj.Z
    return numero_atomico

Testando as funções:

In [7]:
fracoes = calc_fracao_molar("Ba2 Pd4 S8")
print(fracoes)

print(numero_atomico_medio(fracoes))

{'Ba': 0.14285714285714285, 'Pd': 0.2857142857142857, 'S': 0.5714285714285714}
30.285714285714285


Com as bibliotecas e funções definidas e testadas, podemos prosseguir para a próxima etapa: a preparação dos dados.

**Observação:** O objetivo deste notebook também é testar alguns parâmetros para a otimização do modelo. Portanto, existe uma função baseada no `optuna` que será utilizada para encontrar os melhores parâmetros para o modelo. Entretanto, ela será definida apenas quando necessário, para facilitar a compreensão do código.

## 🌐 **Trabalhando com o banco de dados**

Para acessarmos o banco de dados do Materials Project utilizaremos uma **API** *(Application Programming Interface)*. Essa interface `mp-api` permite que usuários acessem e interajam com o vasto banco de dados do Projeto Materials de forma programática, abrindo um leque de possibilidades para análise e exploração de dados. Utilizaremos o ojeto `MPRester`, ele atua como um cliente, permitindo que você se conecte ao banco de dados e realize consultas de forma simples e intuitiva, sem precisar se preocupar com os detalhes da comunicação com a API.

A utilização da API, simplifica o processo de coleta e organização dos dados, permitindo que o foco seja direcionado para a análise e o desenvolvimento do modelo de aprendizado de máquina. Para que possa utiliza-la, é necessário ter uma chave de acesso, que pode ser obtida gratuitamente ao criar uma conta no site do projeto ([The Materials Project](https://next-gen.materialsproject.org/)).

Acessando os dados através da API:

In [8]:
with open ('secrets.json') as secret:
    api_key = json.load(secret)

In [9]:
mpr = MPRester(api_key["api_key"]) 

print(type(mpr)) # Espera-se como resultado 'mp_api.client.mprester.MPRester'

<class 'mp_api.client.mprester.MPRester'>


Agora com o `MPRester` instanciado, podemos realizar consultas ao banco de dados do *Materials Project*. Para isso, utilizaremos um `subrester`, isto se refere a um conjunto de funcionalidades específicas dentro do MPRester que permitem o acesso a diferentes categorias de dados ou a realização de tarefas específicas. Eles são como módulos especializados que organizam as diferentes funcionalidades da API.

Em específico utilizaremos o `summary`, que é como uma função que retorna um resumo conciso das propriedades mais importantes de um material. Ele fornece uma visão geral rápida e eficiente das características principais, sem a necessidade de buscar informações detalhadas em outros subresters. Além disso, é possível especificar filtros e quais propriedades esperamos que a instância retorne, assim como utilizado abaixo [4].

In [10]:
summary_subrester = mpr.materials.summary

resultados = summary_subrester.search(
            has_props = [HasProps.dielectric, HasProps.bandstructure],
            fields = [
        "material_id",         
        "band_gap", 
        "density", 
        "volume",
        "formation_energy",                
        "energy",                  
        "nsites", 
        'elements', 
        'nelements', 
        'composition', 
        'bandstructure',  
        'synthesizable',
        'crystal system',
        'predicted stable',
        'is_gap_direct',
        'ordering',
        'types_of_magnetic_species',
        'total dielectric constant',
        'e_total',
        'theoretical',
        'formation_energy_per_atom',
        'symmetry',
        'density_atomic',
    ]
    )

print(f"O número de materiais retornados é: {len(resultados)}")
print("--Primeiro Material (SummaryDoc)--")
print(resultados[0])

Retrieving SummaryDoc documents:   0%|          | 0/6247 [00:00<?, ?it/s]

O número de materiais retornados é: 6247
--Primeiro Material (SummaryDoc)--
MPDataDoc<SummaryDoc>
nsites=14,
elements=[Element Ba, Element Pd, Element S],
nelements=3,
composition=Composition('Ba2 Pd4 S8'),
volume=316.34140399923814,
density=5.022717317257348,
density_atomic=22.595814571374152,
symmetry=SymmetryData(crystal_system=<CrystalSystem.mono: 'Monoclinic'>, symbol='P2_1/m', number=11, point_group='2/m', symprec=0.1, version='2.0.2'),
material_id=MPID(mp-28967),
formation_energy_per_atom=-1.1240792407142866,
band_gap=0.7791999999999999,
is_gap_direct=False,
bandstructure=BandstructureData(setyawan_curtarolo=BandStructureSummaryData(task_id=MPID(mp-1604877), band_gap=0.7791999999999999, cbm={'band_index': {'1': [54]}, 'kpoint_index': [24, 25], 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint object at 0x000002403CBA8B50>, 'energy': 3.8181, 'projections': {'1': [[0.0, 0.0, 0.0003, 0.0003, 0.0001, 0.0001, 0.0013, 0.0013, 0.0208, 0.0208, 0.0012, 0.0012, 0.0012, 0.0012]

Caso seja necessário avaliar quais opções de pesquisa estão disponíveis, é possível executar o seguinte código:

In [11]:
lista_de_opcoes = mpr.materials.summary.available_fields
print(f"Campos disponíveis para o SummaryDoc são: {lista_de_opcoes}")

Campos disponíveis para o SummaryDoc são: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'bulk_modulus', 'shear_modulus', 'universal_anisotropy', 'homogeneous_poiss

Agora é necessário transformar os resultados obtidos em um DataFrame, para que possamos manipular e analisar os dados de forma mais eficiente. Para isso, sera necessário iterar nos *resultados* obtidos com o `subrester` e então utilizar a biblioteca `pandas`, que possibilita a criação de um *DataFrame* a partir de um dicionário.

In [12]:
data = []
for doc in resultados:
    data.append({
        'material_id': doc.material_id,
        'theoretical': doc.theoretical,    
        "band_gap": doc.band_gap, 
        "density": doc.density, 
        "volume": doc.volume,
        'symmetry': doc.symmetry.crystal_system,
        "dieletric_constant": doc.e_total, 
        'formation_energy_per_atom': doc.formation_energy_per_atom,
        "densidade_atômica": doc.density_atomic,       
        "nsites": doc.nsites,
        'elements': doc.elements, 
        'nelements': doc.nelements, 
        'composition': doc.composition, 
        'bandstructure': doc.bandstructure,
        'is_gap_direct': doc.is_gap_direct,
        'ordering': doc.ordering,
        'types_of_magnetic_species': doc.types_of_magnetic_species,
    })

df = pd.DataFrame(data)
print(df.head())

  material_id  theoretical  band_gap   density      volume      symmetry  \
0    mp-28967        False    0.7792  5.022717  316.341404    Monoclinic   
1   mp-766094         True    2.8980  3.764366  253.915299  Orthorhombic   
2    mp-36577         True    1.7212  3.094976  196.220495     Triclinic   
3  mp-1102092        False    2.0944  2.901260  620.336826    Monoclinic   
4   mp-720391        False    7.4812  1.860992  374.200384  Orthorhombic   

   dieletric_constant  formation_energy_per_atom  densidade_atômica  nsites  \
0           17.048334                  -1.124079          22.595815      14   
1           17.572010                  -3.099174          15.869706      16   
2           18.488667                  -0.766100          28.031499       7   
3            9.596025                  -1.948264          12.923684      48   
4            6.216546                  -1.970766           9.355010      40   

         elements  nelements     composition  \
0     [Ba, Pd, S]   

Durante a implementação do código, percebemos que a leitura direta dos dados para formação do dataframe gerava algum problema para interpretação posterior feita pela máquina. Será possível até perceber diferenças entre o *output* de `.head()` acima e o *output* de `.head()` da célula abaixo (Pode reparar nas coluna *composition* e *elements*).

Infelizmente, não conseguimos compreender a origem do erro para concertá-lo. Entretanto, para contorná-lo, optamos por salvar os dados em um arquivo `.csv` e então realizar a leitura do mesmo para reformação do dataframe, o que solucionou o problema.

In [13]:
df.to_csv('materials.csv')

df = pd.read_csv('materials.csv')

print(df.head(5))

   Unnamed: 0 material_id  theoretical  band_gap   density      volume  \
0           0    mp-28967        False    0.7792  5.022717  316.341404   
1           1   mp-766094         True    2.8980  3.764366  253.915299   
2           2    mp-36577         True    1.7212  3.094976  196.220495   
3           3  mp-1102092        False    2.0944  2.901260  620.336826   
4           4   mp-720391        False    7.4812  1.860992  374.200384   

       symmetry  dieletric_constant  formation_energy_per_atom  \
0    Monoclinic           17.048334                  -1.124079   
1  Orthorhombic           17.572010                  -3.099174   
2     Triclinic           18.488667                  -0.766100   
3    Monoclinic            9.596025                  -1.948264   
4  Orthorhombic            6.216546                  -1.970766   

   densidade_atômica  nsites                                        elements  \
0          22.595815      14             [Element Ba, Element Pd, Element S]  

Com a leitura dos dados funcionando corretamente, podemos prosseguir.

Para nosso trabalho, decidimos utilizar também os elementos que compõem cada material como atributo para o algoritmo de aprendizado de máquinas. Para isso, aplicaremos as funções definidas previamente para extrair propriedades baseadas nos elementos de cada material e adicionar essas informações ao DataFrame. 

Primeiramente, vamos extrair a fração molar de cada elemento presente em um material. A fração molar de um elemento é a razão entre o número de átomos desse elemento e o número total de átomos no material, e, nesse trabalho chamaremos-na de característica *direta*.

In [14]:
df['fractions'] = df['composition'].apply(calc_fracao_molar)

fractions_df = pd.DataFrame(df['fractions'].tolist()).fillna(0)

df = pd.concat([df, fractions_df], axis=1)

Em segunda instância, além da relação direta feita pela fração molar, também é possível extrair informações *derivadas* dos materias, por exemplo, o número atômico médio dos elementos presentes em um material. O número atômico médio é uma medida da média ponderada dos números atômicos dos elementos presentes em um material, levando em consideração a fração molar de cada elemento. 

Essa informação pode facilitar a aplicação de algoritmos, pois gerar frações molares gera muitas colunas, o que pode dificultar a interpretação do modelo. Portanto, o número atômico médio é uma forma de resumir as informações dos elementos presentes em um material em um único valor, facilitando a análise e a interpretação dos dados.

In [15]:
df['average_atomic_number'] = df['fractions'].apply(numero_atomico_medio)

Para continuar, na coluna *symmetry* temos dados categóricos e precisamos transforma-los em númericos, para que o modelo de aprendizado de máquina possa interpretar esses dados.

Utilizaremos o `LabelEncoder` da biblioteca `sklearn.preprocessing` para transformar a coluna categórica em valores numéricos. Esse processo transforma cada valor categórico em um valor inteiro único.

In [16]:
encoder = LabelEncoder()

df['symmetry_encoded'] = encoder.fit_transform(df['symmetry'])

print( df['symmetry_encoded'].unique())

[2 3 5 1 4 6 0]


Por fim, podemos computar algumas características do DataFrame, para isso, utilizaremos as funções `info()`, `isnull().sum()` e visualizaremos algumas colunas com a `head()`.

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6247 entries, 0 to 6246
Columns: 101 entries, Unnamed: 0 to symmetry_encoded
dtypes: bool(2), float64(87), int32(1), int64(3), object(8)
memory usage: 4.7+ MB


In [18]:
df.isnull().sum()

Unnamed: 0               0
material_id              0
theoretical              0
band_gap                 0
density                  0
                        ..
Tb                       0
U                        0
Ac                       0
average_atomic_number    0
symmetry_encoded         0
Length: 101, dtype: int64

In [19]:
df.iloc[:, :20].head()

,Unnamed: 0,material_id,theoretical,band_gap,density,volume,symmetry,dieletric_constant,formation_energy_per_atom,densidade_atômica,nsites,elements,nelements,composition,bandstructure,is_gap_direct,ordering,types_of_magnetic_species,fractions,Ba
0,0,mp-28967,False,0.7792,5.022717,316.341404,Monoclinic,17.048334,-1.124079,22.595815,14,"[Element Ba, Element Pd, Element S]",3,Ba2 Pd4 S8,setyawan_curtarolo=BandStructureSummaryData(ta...,False,NM,[],"{'Ba': 0.14285714285714285, 'Pd': 0.2857142857...",0.142857
1,1,mp-766094,True,2.8980,3.764366,253.915299,Orthorhombic,17.572010,-3.099174,15.869706,16,"[Element F, Element Nb, Element O]",3,Nb4 O8 F4,setyawan_curtarolo=BandStructureSummaryData(ta...,False,NM,[],"{'Nb': 0.25, 'O': 0.5, 'F': 0.25}",0.000000
2,2,mp-36577,True,1.7212,3.094976,196.220495,Triclinic,18.488667,-0.766100,28.031499,7,"[Element As, Element S, Element Sr]",3,Sr1 As2 S4,setyawan_curtarolo=BandStructureSummaryData(ta...,True,NM,[],"{'Sr': 0.14285714285714285, 'As': 0.2857142857...",0.000000
3,3,mp-1102092,False,2.0944,2.901260,620.336826,Monoclinic,9.596025,-1.948264,12.923684,48,"[Element Fe, Element Na, Element O, Element S]",4,Na4 Fe4 S8 O32,setyawan_curtarolo=BandStructureSummaryData(ta...,True,AFM,[Element Fe],"{'Na': 0.08333333333333333, 'Fe': 0.0833333333...",0.000000
4,4,mp-720391,False,7.4812,1.860992,374.200384,Orthorhombic,6.216546,-1.970766,9.355010,40,"[Element B, Element F, Element H, Element N]",4,B4 H16 N4 F16,setyawan_curtarolo=BandStructureSummaryData(ta...,False,NM,[],"{'B': 0.1, 'H': 0.4, 'N': 0.1, 'F': 0.4}",0.000000


Com a importação e preparação dos dados concluída, podemos prosseguir para a etapa de análise exploratória e ajuste dos dados.

## 📈 **Analisando os dados**

A partir de agora pode-se analisar os dados obtidos, para isso utilizaremos a biblioteca `plotly`, uma biblioteca de visualização de dados que permite a criação de gráficos e dashboards interativos, por conta disso é uma ferramenta poderosa para a visualização de dados, oferecendo uma variedade de gráficos e recursos interativos que facilitam a exploração e a interpretação de dados. 

Primeiramente, é possível visualizar a distribuição do *band gap* dos materiais presentes no banco de dados. Para isso, utilizaremos um histograma, um gráfico que representa a distribuição de frequências de um conjunto de dados.

In [20]:
fig = px.histogram(df, x='band_gap', title='Histograma do Band Gap')
fig.add_vline(x=df['band_gap'].mean(), line_dash="dash", line_color="green", annotation_text="Média", annotation_position="top right")
fig.show()

*Gráfico*

Esse gráfico permite visualizar a distribuição dos valores do *band gap* para considerarmos se é realmente um bom target para o nosso modelo. Caso tivessemos uma distribuição muito disbalanceada, poderia significar que nosso dataset não é bom para treinar um modelo, por ser muito enviesado.

Agora é possível visualizar a relação entre o *band gap* e as outras propriedades dos materiais que selecionaremos como atributos. Para isso, utilizaremos um gráfico de dispersão, que utiliza pontos para representar os valores de duas variáveis em um plano cartesiano, permitindo visualizar a relação entre essas variáveis.


In [21]:
fig = make_subplots(rows=2, cols=2, start_cell="top-left", subplot_titles=("Density", "Formation Energy per Atom", "Number of Sites", "Dieletric Constant"))

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['density'], mode='markers', name='density'),
                row=1, col=1)

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['formation_energy_per_atom'], mode='markers', name='formation_energy_per_atom'),
                row=1, col=2)

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['nsites'], mode='markers', name='nsites'),
                row=2, col=1)

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['dieletric_constant'], mode='markers', name='dieletric_constant'),
                row=2, col=2)

fig.update_layout(height=600, width=800, title_text="Matriz de gráficos relacionando o Band gap (x) com os atributos (y)")
fig.show()

Com a matriz de gráficos acima, podemos perceber que há uma aparente relação entre o *band gap* e as propriedades dos materiais que escolhemos. Essa relação sugere que essas propriedades podem ser úteis para prever o *band gap* de materiais, o que justifica a escolha desses atributos para o desenvolvimento do modelo de aprendizado de máquina.

Além disso, também podemos perceber alguns pontos ***outliers***, que podem ser considerados anomalias e devem ser tratados mais adiante.

Por fim, precisamos fazer um gráfico para a relação entre o *band gap* e a conformação cristalina dos materiais, para isso utilizaremos um gráfico boxplot, que é um gráfico que permite visualizar a distribuição de um conjunto de dados, representando a mediana, quartis e *outliers*.

In [22]:
fig = px.box(df, x="symmetry", y="band_gap", title="Boxplot do Band Gap por Conformação")
fig.show()

Pelo gráfico acima, podemos perceber algumas variações no *band gap* de acordo com a conformação cristalina dos materiais, portanto, pode ser plausível considerar essa propriedade como um atributo para o modelo de aprendizado de máquina.

Como dito anteriormente, precisamos tratar os *outliers* presentes nos dados. Para isso, utilizaremos a técnica de **Winsorização**, da biblioteca `scipy.stats`, que consiste em substituir os valores extremos por valores menos extremos, reduzindo a influência desses valores na análise dos dados.

Vantagens da windsorização:
- Preserva a informação dos outliers, mas reduz sua influência nos resultados.
- Mais robusta que a remoção de outliers, especialmente para conjuntos de dados pequenos.
- Pode ser mais adequada que a imputação pela média/mediana para distribuições assimétricas.

In [23]:
df['density'] = winsorize(df['density'], limits=[0.05, 0.05])
df['dieletric_constant'] = winsorize(df['dieletric_constant'], limits=[0.05, 0.05])
df['nsites'] = winsorize(df['nsites'], limits=[0.05, 0.05])
df['formation_energy_per_atom'] = winsorize(df['formation_energy_per_atom'], limits=[0.05, 0.05])

fig = make_subplots(rows=2, cols=2, start_cell="top-left", subplot_titles=("Density", "Formation Energy per Atom", "Number of Sites", "Dieletric Constant"))

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['density'], mode='markers', name='density'),
                row=1, col=1)

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['formation_energy_per_atom'], mode='markers', name='formation_energy_per_atom'),
                row=1, col=2)

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['nsites'], mode='markers', name='nsites'),
                row=2, col=1)

fig.add_trace(go.Scatter(x=df['band_gap'], y=df['dieletric_constant'], mode='markers', name='dieletric_constant'),
                row=2, col=2)

fig.update_layout(height=600, width=800, title_text="Matriz de gráficos relacionando o Band gap (x) com os atributos (y) após winsorização")
fig.show()


Agora percebemos menos *outliers* presentes nos dados, o que pode melhorar a performance do modelo de aprendizado de máquina. Portanto, com os dados trabalhados, podemos prosseguir para as etapas finais. Vale ressaltar que essa operação faz aparecerem gráficos com aspectos "quadradões", mas isso é esperado, pois a técnica de Winsorização altera os valores extremos para valores dentro de um limite.

## 📊 **Testando features**

Para testar a eficácia das features selecionadas, é possível utilizar o `optuna`, uma biblioteca de otimização de hiperparâmetros que permite encontrar os melhores parâmetros para um modelo de aprendizado de máquina. Apesar de normalmente ser utilizada para otimizar hiperparâmetros, neste caso utilizaremos o optuna para verificar o funcionamento de um modelo `RandomForestRegressor` com as features selecionadas, verificando se são funcionais.

Em primeiro caso, iremos aplicar o `optuna` nos dados *derivados* citados anteriormente, para verificar a eficácia das features selecionadas.

In [24]:
def objective_teste1(trial):
    """
    Função objetivo para otimização de hiperparâmetros do RandomForestRegressor.

    Args:
        trial: Objeto Trial do Optuna para gerenciar os hiperparâmetros.

    Returns:
        float: Erro quadrático médio (MSE) nos dados de teste.
    """

    features = ["density", "formation_energy_per_atom", "nsites", 
                "symmetry_encoded", "dieletric_constant", "average_atomic_number", 
                "densidade_atômica"]
    target = "band_gap"
    X = df[features]
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    model = RandomForestRegressor(
        n_estimators=trial.suggest_int("n_estimators", 2, 20),
        max_depth=trial.suggest_int("max_depth", 2, 20),
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)

    return mse

study = optuna.create_study(direction="minimize")
study.optimize(objective_teste1, n_trials=50, show_progress_bar=True)

print(f'Os melhores parâmetros encontrados foram: {study.best_params}')
print(f'O melhor valor encontrado de RMSE foi: {study.best_value}')
print(f'O melhor trial foi: {study.best_trial}')

optuna.visualization.plot_optimization_history(study)

  0%|          | 0/50 [00:00<?, ?it/s]

Os melhores parâmetros encontrados foram: {'n_estimators': 20, 'max_depth': 16}
O melhor valor encontrado de RMSE foi: 1.081536934792846
O melhor trial foi: FrozenTrial(number=9, state=1, values=[1.081536934792846], datetime_start=datetime.datetime(2024, 10, 7, 5, 49, 32, 169592), datetime_complete=datetime.datetime(2024, 10, 7, 5, 49, 32, 579343), params={'n_estimators': 20, 'max_depth': 16}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=20, log=False, low=2, step=1), 'max_depth': IntDistribution(high=20, log=False, low=2, step=1)}, trial_id=9, value=None)


Agora, iremos aplicar o `optuna` nos dados *diretos* citados anteriormente, para verificar a eficácia das features selecionadas.

Vale ressaltar que, pelo grande número de colunas novas adicionadas para fração molar, é mais fácil definir as *features* pelo que não queremos, do que pelo que queremos. Portanto, utilizaremos a função `drop` para retirar as colunas que não queremos.

In [25]:
def objective_teste2(trial):
    """
    Função objetivo para otimização de hiperparâmetros do RandomForestRegressor.

    Args:
        trial: Objeto Trial do Optuna para gerenciar os hiperparâmetros.

    Returns:
        float: Erro quadrático médio (MSE) nos dados de teste.
    """
    #Como utilizaremos muitas colunas de elementos, é melhor trabalhar excluindo as colunas irrelevantes de nossa feature.
    excluded_columns = [
        "band_gap", "material_id", "elements", "composition", "bandstructure",
        "fractions", "symmetry", "is_gap_direct", "ordering", 
        "types_of_magnetic_species"
    ]
    X = df.drop(columns=excluded_columns)
    y = df["band_gap"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    model = RandomForestRegressor(
        n_estimators=trial.suggest_int("n_estimators", 2, 20),
        max_depth=trial.suggest_int("max_depth", 2, 20),
    )

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)

    return mse

study = optuna.create_study(direction="minimize")

study.optimize(objective_teste2, n_trials=50, show_progress_bar=True, )

print(f'Os melhores parâmetros encontrados foram: {study.best_params}')
print(f'O melhor valor encontrado de RMSE foi: {study.best_value}')
print(f'O melhor trial foi: {study.best_trial}')

optuna.visualization.plot_optimization_history(study)

  0%|          | 0/50 [00:00<?, ?it/s]

Os melhores parâmetros encontrados foram: {'n_estimators': 17, 'max_depth': 19}
O melhor valor encontrado de RMSE foi: 0.7005673772752169
O melhor trial foi: FrozenTrial(number=24, state=1, values=[0.7005673772752169], datetime_start=datetime.datetime(2024, 10, 7, 5, 49, 56, 839781), datetime_complete=datetime.datetime(2024, 10, 7, 5, 49, 57, 704394), params={'n_estimators': 17, 'max_depth': 19}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=20, log=False, low=2, step=1), 'max_depth': IntDistribution(high=20, log=False, low=2, step=1)}, trial_id=24, value=None)


Pelos resultados acima, podemos notar que aparentemente as features selecionadas são eficazes para a previsão do *band gap* dos materiais, uma vez que o modelo obteve um desempenho razoável. Além disso, a seleção *direta* apresentou um desempenho ligeiramente superior ao da seleção *derivada*, indicando uma preferência de uso. Entretanto isso pode variar de acordo com o modelo utilizado.

Por fim, pode-se escrever o conjunto de dados como um arquivo excel, para que possa ser utilizado em outros trabalhos, assim como ocorrerá nos próximos notebooks.

In [26]:
df.to_csv('materials_trabalhado.csv')

## ⚔️ **Conclusão**

Neste caderno conseguimos realizar com eficácia a coleta e organização dos dados do *Materials Project*, utilizando a API `mp-api` e o objeto `MPRester`. A extração de features baseadas nos elementos presentes em cada material permitiu enriquecer o conjunto de dados e fornecer informações valiosas para o modelo de aprendizado de máquina. 

Além disso, a análise exploratória dos dados revelou insights importantes sobre as relações entre as features e o *band gap*, guiando a seleção de features para o modelo, principalmente com o teste utilizando o `optuna`.

## 📖 **Referências Bibliográficas**

1. MATOS, Fernando Barbosa. **Modelagem Computacional do Comportamento de Células Fotovoltaicas Baseado nas Propriedades Físicas dos Materiais.** 2006. 125 f. Dissertação (Mestrado em Engenharia Elétrica) - Universidade Federal de Uberlândia, Uberlândia, 2006.

2. JAIN, A. et al. **Commentary: The Materials Project: A materials genome approach to accelerating materials innovation.** APL Materials, v. 1, n. 1, p. 011002, jul. 2013.

3. MATERIALSPROJECT. **GitHub - materialsproject/api: New API client for the Materials Project**. Disponível em: <https://github.com/materialsproject/api>. Acesso em: 7 out. 2024. 

4. e**mmet/emmet-core/emmet/core/summary.py at 1852752d6d283f2a16fb2e0a77c6db65393d15b0 · materialsproject/emmet.** Disponível em: <https://github.com/materialsproject/emmet/blob/1852752d6d283f2a16fb2e0a77c6db65393d15b0/emmet-core/emmet/core/summary.py#L17-L42>. Acesso em: 7 out. 2024. 

#### **Documentações:**

Este projeto utilizou as seguintes bibliotecas:

**Bibliotecas Python:**

* **`json`:** [https://docs.python.org/3/library/json.html](https://docs.python.org/3/library/json.html) - Manipulação de arquivos JSON.
* **`os`:** [https://docs.python.org/3/library/os.html](https://docs.python.org/3/library/os.html) - Interação com o sistema operacional.
* **`re`:** [https://docs.python.org/3/library/re.html](https://docs.python.org/3/library/re.html) -  Operações com expressões regulares.

**Bibliotecas de Terceiros:**

* **`optuna`:** [https://optuna.readthedocs.io/en/stable/](https://optuna.readthedocs.io/en/stable/) - Otimização de hiperparâmetros.
* **`pandas`:** [https://pandas.pydata.org/docs/](https://pandas.pydata.org/docs/) - Manipulação e análise de dados.
* **`plotly.express`:** [https://plotly.com/python/plotly-express/](https://plotly.com/python/plotly-express/) - Criação de gráficos interativos.
* **`plotly.graph_objects`:** [https://plotly.com/python/graph-objects/](https://plotly.com/python/graph-objects/) - Criação de gráficos com maior controle.
* **`plotly.subplots`:** [https://plotly.com/python/subplots/](https://plotly.com/python/subplots/) - Criação de múltiplos gráficos em uma figura.
* **`emmet.core.summary`:**  - Biblioteca para análise de materiais[3].
* **`mp_api.client`:**  - Interface para a API do Materials Project[3]. 
* **`pymatgen.core.periodic_table`:** [https://pymatgen.org/](https://pymatgen.org/) - Acesso à tabela periódica e propriedades dos elementos.
* **`scipy.stats.mstats.winsorize`:** [https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.winsorize.html) -  Função para windsorização (tratamento de outliers).
* **`sklearn.ensemble`:** [https://scikit-learn.org/stable/modules/ensemble.html](https://scikit-learn.org/stable/modules/ensemble.html) - Algoritmos de ensemble learning.
* **`sklearn.metrics`:** [https://scikit-learn.org/stable/modules/model_evaluation.html](https://scikit-learn.org/stable/modules/model_evaluation.html) - Métricas para avaliação de modelos.
* **`sklearn.model_selection`:** [https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html) - Ferramentas para seleção e avaliação de modelos.
* **`sklearn.preprocessing`:** [https://scikit-learn.org/stable/modules/preprocessing.html](https://scikit-learn.org/stable/modules/preprocessing.html)- Pré-processamento de dados.


## 👣 **Próximos passos**

Este é o primeiro de uma série de 4 notebooks que compõem o trabalho final da disciplina de Aprendizado de Máquina. Nos próximos cadernos, serão realizadas as etapas de desenvolvimento e avaliação de modelos de aprendizado de máquina para prever o *band gap* de materiais, e a otimização dos modelos para obter a melhor performance. Para acompanhar o desenvolvimento do projeto, acesse os próximos [notebooks](https://github.com/CaioRuas24010/SepulcroDeDelfos/tree/main/A%20Batalha%20Contra%20Dragao):

1. **[`introducao.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/introducao.ipynb) - Baixando e organizando os dados**
2. **[`modelo_dos_k-nn_vizinhos.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/modelo_dos_k-nn_vizinhos.ipynb) - Estudando o modelo k-NN**
3. **[`arvore_de_decisao.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/arvore_de_decisao.ipynb) - Estudando o modelo Árvore de Decisão**
4. **[`conclusao.ipynb`](https://github.com/CaioRuas24010/SepulcroDeDelfos/blob/main/A%20Batalha%20Contra%20Dragao/conclusao.ipynb) - Resultados e discussões finais** 

---

# **<span style="font-family: 'Palatino Linotype', serif;">A jornada continua...</span>**

*<span style="font-family: 'Angilla Tattoo'"> <br> E assim, com os mapas estelares decifrados e as runas ancestrais traduzidas, os bravos guerreiros concluem sua preparação. <br> <br> Mas a jornada apenas começa. O caminho à frente é longo e repleto de perigos. A primeira parada será o misterioso vilarejo de vizinhos, onde a sabedoria ancestral dos KNN, os guardiões da  profecia, poderá revelar a localização do covil do dragão. Lá, nossos heróis enfrentarão o primeiro desafio, usando a força da união e o poder da similaridade para desvendar os segredos ancestrais e se aproximar da vitória. <br> <br> Que os deuses ancestrais os abençoem nesta jornada! Avante, heróis, rumo ao destino e à glória! <br> <br> Nossos algoritmos são oráculos, nossos dados são ossos ancestrais. <br> Sepulcro de Delfos </span>*